In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain_chroma import Chroma
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain_core.tools import Tool
# you can you other types of database like postgres rather than in memeory
import sys
import os
import operator
import numpy
# Add parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.path.dirname('apple_case'), '..'))
sys.path.insert(0, parent_dir)
# from dotenv import load_dotenv
import json
# _ = load_dotenv()
from src.utils import *

In [4]:
# Build the path to config.json
config_path = os.path.join(parent_dir, 'src', 'config.json')

with open(config_path, 'r') as f:
    config = json.load(f)

In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [6]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        )


  # #Place vectorDB under /tmp. It can be anywhere else
# # from langchain.vectorstores import Chroma
persist_directory = config["vectorstore"]["persist_directory"]
products_collection_name = config["vectorstore"]["products_collection_name"]  
vectordb = Chroma(collection_name=products_collection_name, embedding_function=embeddings,
                            persist_directory=persist_directory)


print(vectordb._collection.count())

education_collection_name = config["vectorstore"]["education_collection_name"]  
education_vectordb = Chroma(collection_name=education_collection_name, embedding_function=embeddings,
                            persist_directory=persist_directory)

inventory_collection_name = config["vectorstore"]["inventory_collection_name"]  
inventory_vectordb = Chroma(collection_name=inventory_collection_name, embedding_function=embeddings,
                            persist_directory=persist_directory)


# # Initialize ChatOpenAI
# model_name = "gpt-4o-mini"
# llm = ChatOpenAI(model_name=model_name)

# # define prompts
# prompt = PromptTemplate(template=products_template, input_variables=["context", "question"])
prompt2 = PromptTemplate(template=education_template, input_variables=["context", "question"])
prompt3 = PromptTemplate(template=inventory_template, input_variables=["context", "question"])

     
      

        

0


In [7]:
class Agent:
    def __init__(self, model, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.model = model
        
        self.education_chain = RetrievalQA.from_chain_type(
            self.model, retriever=education_vectordb.as_retriever(), chain_type_kwargs={"prompt": prompt2}
        )

        self.inventory_chain = RetrievalQA.from_chain_type(
            self.model, retriever=inventory_vectordb.as_retriever(), chain_type_kwargs={"prompt": prompt3
            }
        )

        #      # Define tools properly
        tools = [
            # Tool(
            #     name="Apple Sales Assistant",
            #     func=self.rag_chain.run,
            #     description="useful for answering questions about apple products recommendations including pricing, tech specs and descriptions."
            # ),
            Tool(
                name="apple_education_assistant",
                func=self.education_chain.run,
                description="useful for answering questions about how and where to buy Apple products."
            ),
            Tool(
                name="apple_inventory_assistant",
                func=self.inventory_chain.run,
                description="useful for checking on the inventory of products."
            ),
        ]

       
        self.tools = {tool.name: tool for tool in tools}
        self.model = model.bind_tools(tools)

   
       

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        tool_messages = []
    
        for t in tool_calls:
            tool_name = t['name']
            tool_args = t['args']
            tool_call_id = t['id']
    
            print(f"Calling tool: {tool_name} with args: {tool_args}")
    
            result = self.tools[tool_name].invoke(tool_args)
    
            tool_messages.append(ToolMessage(tool_call_id=tool_call_id, name=tool_name, content=str(result)))
    
        return {'messages': tool_messages}

In [8]:
def echo_tool_fn(input: str) -> str:
    return f"You said: {input}"

tool = Tool.from_function(
    func=echo_tool_fn,
    name="echo",
    description="Echoes what the user says."
)


In [9]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
# you can you other types of database like postgres rather than in memeory
conn = sqlite3.connect(":memory:", check_same_thread=False)  
memory = SqliteSaver(conn)

In [10]:
memory

In [11]:
prompt = """You are a Apple store assistant. Use the apple education assistant for answering about product buying strategies or where to buy apple products, or use the apple inventory assistant for shopping queries. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
# model = ChatOpenAI(model="gpt-4o-mini")

model = init_chat_model("gpt-4o-mini", model_provider="openai")
abot = Agent(model,  checkpointer=memory, system=prompt)
# agent = model.bind_tools([tool])

In [12]:
abot.tools

{'apple_education_assistant': Tool(name='apple_education_assistant', description='useful for answering questions about how and where to buy Apple products.', func=<bound method Chain.run of RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\nYou are an Apple products educator, users will ask you questions on how to get the best deals for apple products or comparisons between different products. Use the following piece of context to answer the question.\n\nOnly answer questions related to product comparisons and specifications, buying strategies, deals and discounts.\nIf you don't know the answer, just say you don't know.\n\nKeep your replies concise and limited to 4 sentences. \n\nContext: {context}\nQuestion: {question}\nAnswer:\n"), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Compl

In [13]:
messages = [HumanMessage(content="Do you have iphones in stock")]

In [14]:
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SWMNsK8JKnrSGVXKIX9LDiRb', 'function': {'arguments': '{"__arg1":"iphone"}', 'name': 'apple_inventory_assistant'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 183, 'total_tokens': 201, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BmuHRGx5pQlnvr4vOKTeIX2BlS4rw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--90d8a872-e7cb-4c03-a990-4ba7ad845351-0', tool_calls=[{'name': 'apple_inventory_assistant', 'args': {'__arg1': 'iphone'}, 'id': 'call_SWMNsK8JKnrSGVXKIX9LDiRb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 183, 'output_tokens': 18, 